# 0.0. Info Table

Monster Hunter 2 Dos Database App

    1. Lista de Quests separadas por Ranks.

In [8]:
import re
import requests
import sqlite3
import pandas as pd

from bs4 import BeautifulSoup
from sqlalchemy import create_engine

# 1.0. Quest List Data

## 1.1. Jp Quest Train

In [241]:
soup = BeautifulSoup( open('../data/page.html').read(), 'html.parser' )

quest_training = [list(filter(None, p.get_text().split('\n'))) for p in soup.find_all( 'table', class_='cnt-tb2' )[-1].find_all('tr')]

### 1.1.0. Jp Quest Cleaning

In [242]:
df_train = pd.DataFrame( quest_training )

# Rename and Filter Columns
df_train.columns = ['id', 'rank', 'name']+[p.replace(' ', '_').lower() for p in df_train.iloc[0, :].tolist()][3:]
df_train = df_train.drop( columns=['season', 'hrp', 'special_conditions', 'contract_money'], axis=1 )
df_train.columns = ['id', 'rank', 'name', 'reward', 'time', 'area', 'request_details']
df_train = df_train.iloc[1:, :].reset_index( drop=True )

# Individual Columns
df_train['rank'] = df_train['rank'].str.replace('¡ù£±', '1')
df_train['rank'] = df_train['rank'].str.replace('¡ù£²', '2')
df_train['rank'] = df_train['rank'].str.replace('¡ù£³', '3')
df_train['rank'] = df_train['rank'].str.replace('¡ù£´', '4')
df_train['rank'] = df_train['rank'].str.replace('¡ù£µ', '5')
df_train['time'] = df_train['time'].str.extract('(\d+)')[0]
df_train['area'] = df_train['area'].apply( lambda x: x.title() )
df_train['area'] = df_train['area'].str.replace('Dense Forest', 'Jungle')

df_train['name'] = df_train['name'].str.replace('¢ä', '')
df_train['name'] = df_train['name'].str.replace('¢ã', '')
df_train['name'] = [p.lower().replace('«basic hunter» ', '')
                     .replace('<< basics of hunter >> ', '').strip().title() for p in df_train['name'].tolist()]

# Generate New Rewards
reward = [re.findall( '\d+', p ) for p in df_train['reward'].tolist()]

# full_reward, main_reward, sub_a_reward, sub_b_reward
df_train['main_reward']  = [p[0]+'z' for p in reward]
df_train['full_reward']  = [str(int(p[0]) + int(p[1]) + int(p[2]))+'z' for p in reward]
df_train['sub_a_reward'] = [p[1]+'z' for p in reward]
df_train['sub_b_reward'] = [p[2]+'z' for p in reward]

df_train = df_train.drop( columns=['reward'], axis=1)

# Change Dtypes
df_train['rank'] = df_train['rank'].astype('int64')
df_train['time'] = df_train['time'].astype('int64')

# Split Request Details
df_train['request_details'] = df_train['request_details'].str.replace('Delivering', 'Delivery')
df_train['request_details'] = df_train['request_details'].str.replace(' Deliver ', 'Delivery ')
df_train['request_details'] = [j[2:] if j.startswith('/') else j for j in 
                                 [p.replace('None',       '/ None')
                                   .replace('Delivery',   '/ Delivery')
                                   .replace('Subjugation','/ Subjugation ')
                                   .replace('Hunting',    '/ Hunting') for p in df_train['request_details'].tolist()]]

df_aux = df_train['request_details'].str.split('/ ', expand=True)

df_aux.iloc[29, 1]  = 'Delivery of 8 ice crystals'
df_aux.iloc[29, 2]  = 'Hunt 1 Dodo Blanco'
df_aux.iloc[23, -1] = 'Delivery of 10 sliced cactus'

df_aux[0] = df_aux[0].str.replace( 'Ranposu 5 horses subdue of ', 'Hunt 5 Velociprey' )
df_aux[0] = df_aux[0].apply( lambda x: x.strip() )

for i in range( len( df_aux.columns ) ):
    df_aux[i] = df_aux[i].apply( lambda x: x.strip().replace('None', 'No Sub') if pd.notnull(x) else x )
    df_aux[i] = df_aux[i].fillna('No Sub') 
    
df_aux.columns = ['main', 'sub_a', 'sub_b']

df_train = pd.concat([df_train, df_aux], axis=1)
df_train = df_train.drop( columns=['request_details'], axis=1 )
df_train = df_train[['id', 'rank', 'name', 'time', 'area', 'main', 'main_reward', 
                     'full_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward']]

# Script Verify
df_train['name']  = [p.replace(' Of', '') if p.endswith('Of') else p for p in df_train['name'].tolist()]
cols = ['name', 'main', 'sub_a', 'sub_b']
df_train.iloc[0, 2] = 'Hunter Basics'
df_train.loc[24, 'main']  = 'Hunt Rio Leia'
df_train.loc[24, 'sub_a'] = 'Delivery of 1 hunting flying dragon egg'
df_train.loc[24, 'sub_b'] = 'Delivery of 4 Kireasi'
for i in cols:
    df_train[i] = [p.replace('of', '') if p.endswith('of') else p for p in df_train[i].tolist()]
    df_train[i] = df_train[i].apply( lambda x: x.replace( 'Subjugation ', 'Hunt' )
                                                .replace('subjugation of', 'Hunt')
                                                .replace('subjugation', 'Hunt')
                                                .replace('dragon', '') )

# 2.0. En Quest Village

## 2.1. En Data Collect

In [9]:
url = 'https://monsterhunter.fandom.com/wiki/MH2:_Quests'
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

soup = BeautifulSoup( requests.get( url, hdr ).text, 'html.parser' )

### 2.1.0. One Example Collect

In [90]:
list_quests = soup.find_all('table', {'style': "border: 3px solid #007d11; background-color:#AC9A7E; color:#371c01; width:100%;"})

# 'rank', 'name', 'time', 'area', 'main', 'main_reward', 'full_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward'
q_name  = list_quests[0].find('span', class_="mw-headline").get_text()
q_time  = list_quests[0].find_all('p')[2].get_text().split('\n')[0]
q_main  = list_quests[0].find_all('p')[1].get_text()
q_m_rw  = list_quests[0].find_all('p')[2].get_text().split('\n')[1]
q_sub_a = list_quests[0].find_all('td', attrs={'colspan': '2'} )[1].get_text()
q_sub_b = list_quests[0].find_all('td', attrs={'colspan': '2'} )[1].get_text()
sub_a_rw = list_quests[0].find_all('p')[2].get_text().split('\n')[2]
sub_b_rw = list_quests[0].find_all('p')[2].get_text().split('\n')[3]

### 2.1.1. Multiple Index on One Color 

In [97]:
df = pd.DataFrame( columns=['name', 'time', 'main', 'main_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward'] )

for i in range( len( list_quests ) ):
    list_quests = soup.find_all('table', {'style': "border: 3px solid #007d11; background-color:#AC9A7E; color:#371c01; width:100%;"})

    q_name  = list_quests[i].find('span', class_="mw-headline").get_text()
    q_time  = list_quests[i].find_all('p')[2].get_text().split('\n')[0]
    q_main  = list_quests[i].find_all('p')[1].get_text()
    q_m_rw  = list_quests[i].find_all('p')[2].get_text().split('\n')[1]
    q_sub_a = list_quests[i].find_all('td', attrs={'colspan': '2'} )[1].get_text()
    q_sub_b = list_quests[i].find_all('td', attrs={'colspan': '2'} )[1].get_text()
    sub_a_rw = list_quests[i].find_all('p')[2].get_text().split('\n')[2]
    sub_b_rw = list_quests[i].find_all('p')[2].get_text().split('\n')[3]
    
    df_aux = pd.DataFrame( [q_name, q_time, q_main, q_m_rw, q_sub_a, q_sub_b, sub_a_rw, sub_b_rw] ).T
    df_aux.columns = ['name', 'time', 'main', 'main_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward']
    
    df = pd.concat( [df, df_aux], axis=0 )
    

### 2.1.2. Multiple Colors 

In [232]:
color_list = ['#007d11;', '#a57564;', '#53305C;', '#8C8F83;', '#762622;', '#93C72C;', '#8a7518;']
df = pd.DataFrame( columns=['name', 'time', 'area', 'season', 'main', 'main_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward'] )

for p in color_list:
    list_quests = soup.find_all('table', {'style': "border: 3px solid " + p + " background-color:#AC9A7E; color:#371c01; width:100%;"})
    list_area   = soup.find_all('table', {'style': "border: 3px solid " + p[1:] + " background-color:#EBEBEB; color:#371c01; width:100%;"})
    
    for i in range( len( list_quests ) ):
        # 'rank', 'name', 'time', 'area', 'season', 'main', 'main_reward', 'full_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward'
        #q_rank = 
        q_name  = list_quests[i].find('span', class_="mw-headline").get_text()
        q_time  = list_quests[i].find_all('p')[2].get_text().split('\n')[0]
        q_area  = list_area[i].get_text().replace('\n', '')
        q_seasn = list_quests[i].find_all('a', class_='image')[0]['title']
        q_main  = list_quests[i].find_all('p')[1].get_text()
        q_m_rw  = list_quests[i].find_all('p')[2].get_text().split('\n')[1]
        #q_f_rw  = 
        q_sub_a = list_quests[i].find_all('td', attrs={'colspan': '2'} )[1].get_text()
        q_sub_b = list_quests[i].find_all('td', attrs={'colspan': '2'} )[1].get_text()
        sub_a_rw = list_quests[i].find_all('p')[2].get_text().split('\n')[2]
        sub_b_rw = list_quests[i].find_all('p')[2].get_text().split('\n')[3]

        df_aux = pd.DataFrame( [q_name, q_time, q_area, q_seasn, q_main, q_m_rw, q_sub_a, q_sub_b, sub_a_rw, sub_b_rw] ).T
        df_aux.columns = ['name', 'time', 'area', 'season', 'main', 'main_reward', 'sub_a', 'sub_b', 'sub_a_reward', 'sub_b_reward']

        df = pd.concat( [df, df_aux], axis=0 )

## 2.1. En Data Cleaning

In [235]:
df_village = df.copy()
df_village = df_village.reset_index( drop=True )

# Individual Columns
df_village['time']   = df_village['time'].str.extract('(\d+)')[0].astype('int64')
df_village['area']   = df_village['area'].str.extract('([a-zA-Z ]+)')[0]
df_village['season'] = df_village['season'].str.replace(' Season', '')
df_village['main']   = df_village['main'].apply( lambda x: x.replace('\n', '').replace('(特産キノコ)', ''))
df_village['main']   = df_village['main'].apply( lambda x: x.replace('\n', '').replace('(特産キノコ)', ''))
df_village['sub_a']  = df_village['sub_a'].apply( lambda x: x.replace('\n', '').replace('N/A', 'No Sub'))
df_village['sub_b']  = df_village['sub_b'].apply( lambda x: x.replace('\n', '').replace('N/A', 'No Sub'))

# Generate New Reward
df_village['full_reward'] = [str(+df_village['main_reward'].str.extract('(\d+)')[0].astype('int64').tolist()[p]
                                 +df_village['sub_a_reward'].str.extract('(\d+)')[0].astype('int64').tolist()[p]
                                 +df_village['sub_b_reward'].str.extract('(\d+)')[0].astype('int64').tolist()[p])+'z'
                                 for p in range(len(df_village))]

df_village['main_reward']  = [p+'z' for p in df_village['main_reward'].str.extract('(\d+)')[0].tolist()]
df_village['sub_a_reward'] = [p+'z' for p in df_village['sub_a_reward'].str.extract('(\d+)')[0].tolist()]
df_village['sub_b_reward'] = [p+'z' for p in df_village['sub_b_reward'].str.extract('(\d+)')[0].tolist()]

# Generate Rank Columns with individual index
df_village['rank'] = 1
df_village.iloc[10:16, -1] = 2
df_village.iloc[16, -1] = 3
df_village.iloc[17, -1] = 4
df_village.iloc[20:23, -1] = 2
df_village.iloc[23:29, -1] = 3
df_village.iloc[36:40, -1] = 2 
df_village.iloc[40:43, -1] = 3
df_village.iloc[43:46, -1] = 4
df_village.iloc[47:49, -1] = 2
df_village.iloc[49:51, -1] = 3
df_village.iloc[51:53, -1] = 4
df_village.iloc[57:60, -1] = 3
df_village.iloc[60:62, -1] = 4
df_village.iloc[62:65, -1] = 2
df_village.iloc[65, -1]  = 4
df_village.iloc[67:, -1] = 4
df_village.iloc[66, -1]  = 3

# Select Columns
df_village = df_village[['rank', 'name', 'time', 'season', 'area', 'main', 'main_reward', 'full_reward','sub_a',
                         'sub_b', 'sub_a_reward', 'sub_b_reward']]

# 3.0. Quest List Storange

In [254]:
con = sqlite3.connect('../sql_databases/quests.sqlite')
c   = con.cursor()

query_create_table = '''
    CREATE TABLE train_quests (
        id              TEXT,
        rank            INTEGER,
        name            TEXT,
        time            INTEGER,
        area            TEXT,
        main            TEXT,
        main_reward     TEXT,
        full_reward     TEXT,
        sub_a           TEXT,
        sub_b           TEXT,
        sub_a_reward    TEXT,
        sub_b_reward    TEXT );

'''

c.execute(query_create_table)
con.commit()

db  = create_engine( 'sqlite:///../sql_databases/quests.sqlite' )
con = db.connect()

df_village.to_sql( 'village_quests', con=con, if_exists='append', index=False )
df_train.to_sql( 'train_quests', con=con, if_exists='append', index=False )